### FASE 5 da Libertadores - Final

In [1]:
import cartolafc
import pandas as pd
from difflib import get_close_matches
import json
import os

pd.set_option('display.max_columns', 50)            # permite a visualiza√ß√£o de 50 colunas do dataframe
pd.options.display.float_format = '{:.2f}'.format   # pandas: para todos os n√∫meros aparecerem com duas casas decimais

# Cria uma inst√¢ncia da API
api = cartolafc.Api(attempts=5)

2025-08-12 23:01:20,285 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


### Carregar o arquivo Excel com os classificados e Gerar um Dicion√°rio

In [2]:
CAMINHO_CLASSIFICADOS = "classificados_fase_4.xlsx"
CAMINHO_PERDEDORES = "perdedores_fase_4.xlsx"

# Inicializa√ß√£o segura
nomes_por_id = {}
dados_torneio_semi = []

# Verificar se ambos os arquivos existem
if not os.path.exists(CAMINHO_CLASSIFICADOS) or not os.path.exists(CAMINHO_PERDEDORES):
    print("üìÅ Aguardando os arquivos de classificados e perdedores serem gerados.")
else:
    try:
        # Carregar arquivos
        df_classificados = pd.read_excel(CAMINHO_CLASSIFICADOS)
        df_perdedores = pd.read_excel(CAMINHO_PERDEDORES)

        # Verifica√ß√µes b√°sicas
        if df_classificados.empty or df_perdedores.empty:
            print("üìÑ Um dos arquivos existe mas est√° vazio.")
        else:
            # Filtrar apenas linhas v√°lidas
            df_validos = df_classificados[df_classificados["classificado_id"].notnull()].reset_index(drop=True)
            df_perdedores_validos = df_perdedores[df_perdedores["perdedor_id"].notnull()].reset_index(drop=True)

            if len(df_validos) < 2 or len(df_perdedores_validos) < 2:
                print("‚ö†Ô∏è Aguardando mais confrontos. Ainda n√£o h√° dados suficientes para montar a final e disputa de 3¬∫ lugar.")
            else:
                # Gerar dicion√°rio nome ‚Üí ID (apenas dos classificados, mas voc√™ pode expandir)
                nomes_por_id = dict(zip(df_validos["classificado_id"], df_validos["classificado_nome"]))
                
                # Expandir nomes_por_id com perdedores tamb√©m
                nomes_perdedores = dict(zip(df_perdedores_validos["perdedor_id"], df_perdedores_validos["perdedor_nome"]))

                display(nomes_por_id, nomes_perdedores)
                print("\n" + "-" * 100 + "\n")

                # Definir confrontos da fase final
                dados_torneio_semi = [
                    ("Final", df_validos.loc[0, "classificado_id"]),
                    ("Final", df_validos.loc[1, "classificado_id"]),
                    ("Decis√£o 3¬∫ Lugar", df_perdedores_validos.loc[0, "perdedor_id"]),
                    ("Decis√£o 3¬∫ Lugar", df_perdedores_validos.loc[1, "perdedor_id"]),
                ]

                print("‚úÖ Confrontos da fase final definidos:")
                for linha in dados_torneio_semi:
                    print(linha)

    except Exception as e:
        print(f"‚ùå Erro ao processar os arquivos: {e}")


{44574236: 'Analove10 ITAQUI GRANDE!!', 50158297: 'L√° do Itaqui'}

{3246608: 'Real SCI', 1273719: 'Texas Club 2025'}


----------------------------------------------------------------------------------------------------

‚úÖ Confrontos da fase final definidos:
('Final', 44574236)
('Final', 50158297)
('Decis√£o 3¬∫ Lugar', 3246608)
('Decis√£o 3¬∫ Lugar', 1273719)


In [3]:
display(dados_torneio_semi)

[('Final', 44574236),
 ('Final', 50158297),
 ('Decis√£o 3¬∫ Lugar', 3246608),
 ('Decis√£o 3¬∫ Lugar', 1273719)]

In [4]:
# Criar DataFrame base com dados dos confrontos finais
df_dados_torneio = pd.DataFrame(dados_torneio_semi, columns=["Jogo", "ID do Time"])

# # Adicionar Nome do Time usando o dicion√°rio de nomes_por_id
# df_dados_torneio["Nome do Time"] = df_dados_torneio["ID do Time"].map(nomes_por_id)

# Unir os dois dicion√°rios antes do .map()
nomes_completos = nomes_por_id.copy()
nomes_completos.update(nomes_perdedores)

df_dados_torneio["Nome do Time"] = df_dados_torneio["ID do Time"].map(nomes_completos)

# Verifica√ß√£o adicional caso o nome n√£o seja encontrado
df_dados_torneio["Nome do Time"] = df_dados_torneio["Nome do Time"].fillna("Desconhecido")

# Adicionar ID no Grupo no formato 1_F, 2_F, 1_D, 2_D etc.
df_dados_torneio["ID no Grupo"] = df_dados_torneio.groupby("Jogo").cumcount() + 1
df_dados_torneio["ID no Grupo"] = df_dados_torneio["ID no Grupo"].astype(str) + "_" + df_dados_torneio["Jogo"].str[0]

# Reorganizar colunas
df_torneio_semi_liberta = df_dados_torneio[["Jogo", "ID do Time", "Nome do Time", "ID no Grupo"]]

# Separar confrontos por jogo
df_liberta_jogo_1 = df_torneio_semi_liberta[df_torneio_semi_liberta["Jogo"] == "Final"]
df_liberta_jogo_2 = df_torneio_semi_liberta[df_torneio_semi_liberta["Jogo"] == "Decis√£o 3¬∫ Lugar"]

# Criar estrutura de grupos
grupos = {
    "Final": df_liberta_jogo_1,
    "Decis√£o 3¬∫ Lugar": df_liberta_jogo_2
}

# Exibir os dois DataFrames
display(df_liberta_jogo_1, df_liberta_jogo_2)

display(df_dados_torneio)


Jogo  ID do Time               Nome do Time ID no Grupo
0  Final    44574236  Analove10 ITAQUI GRANDE!!         1_F
1  Final    50158297               L√° do Itaqui         2_F

Jogo  ID do Time     Nome do Time ID no Grupo
2  Decis√£o 3¬∫ Lugar     3246608         Real SCI         1_D
3  Decis√£o 3¬∫ Lugar     1273719  Texas Club 2025         2_D

Jogo  ID do Time               Nome do Time ID no Grupo
0             Final    44574236  Analove10 ITAQUI GRANDE!!         1_F
1             Final    50158297               L√° do Itaqui         2_F
2  Decis√£o 3¬∫ Lugar     3246608                   Real SCI         1_D
3  Decis√£o 3¬∫ Lugar     1273719            Texas Club 2025         2_D

### Defini√ß√£o dos Confrontos da Rodada da Fase 5 da Libertadores (Rodada 17)

In [5]:
# Rodada 1 - Fase 5 Libertadores (Equivalente a 17¬∫ Rodada do Campeonato Brasileiro) 
confrontos_1a_rodada = [
    # Final
    ("Final", "1_F", "2_F"),    

    # Decis√£o do 3¬∫ lugar
    ("Decis√£o 3¬∫ Lugar", "1_D", "2_D")      
]


In [6]:
# Transformar em DataFrame
df_confrontos = pd.DataFrame(confrontos_1a_rodada, columns=["Jogo", "Mandante_ID", "Visitante_ID"])

# Junta com df_torneio para buscar dados dos mandantes
df_mandantes = df_torneio_semi_liberta.rename(columns={
    "ID no Grupo": "Mandante_ID",
    "Nome do Time": "Mandante_Nome",
    "ID do Time": "Mandante_ID_Time"
})[["Jogo", "Mandante_ID", "Mandante_Nome", "Mandante_ID_Time"]]

# Junta com df_torneio para buscar dados dos visitantes
df_visitantes = df_torneio_semi_liberta.rename(columns={
    "ID no Grupo": "Visitante_ID",
    "Nome do Time": "Visitante_Nome",
    "ID do Time": "Visitante_ID_Time"    
})[["Jogo", "Visitante_ID", "Visitante_Nome", "Visitante_ID_Time"]]

display(df_confrontos)

Jogo Mandante_ID Visitante_ID
0             Final         1_F          2_F
1  Decis√£o 3¬∫ Lugar         1_D          2_D

In [7]:
# Transformar em DataFrame
df_confrontos = pd.DataFrame(confrontos_1a_rodada, columns=["Jogo", "Mandante_ID", "Visitante_ID"])
df_confrontos["Rodada"] = 17
df_rodada_15 = df_confrontos.merge(df_mandantes, on=["Jogo", "Mandante_ID"])
df_rodada_15 = df_rodada_15.merge(df_visitantes, on=["Jogo", "Visitante_ID"])

display(df_rodada_15)

Jogo Mandante_ID Visitante_ID  Rodada  \
0             Final         1_F          2_F      17   
1  Decis√£o 3¬∫ Lugar         1_D          2_D      17   

               Mandante_Nome  Mandante_ID_Time   Visitante_Nome  \
0  Analove10 ITAQUI GRANDE!!          44574236     L√° do Itaqui   
1                   Real SCI           3246608  Texas Club 2025   

   Visitante_ID_Time  
0           50158297  
1            1273719

In [8]:
df_rodadas = pd.concat([
    df_rodada_15
], ignore_index=True)

# Ajustar a numera√ß√£o da rodada para refletir as rodadas do Cartola (Rodada 17)
df_rodadas["Rodada"] = df_rodadas["Rodada"]

df_rodadas.to_excel("confrontos_fase_5_libertadores.xlsx", index=False)

# Exibir os confrontos da fase 5 (Final)
display(df_rodadas.head(8)) 

Jogo Mandante_ID Visitante_ID  Rodada  \
0             Final         1_F          2_F      17   
1  Decis√£o 3¬∫ Lugar         1_D          2_D      17   

               Mandante_Nome  Mandante_ID_Time   Visitante_Nome  \
0  Analove10 ITAQUI GRANDE!!          44574236     L√° do Itaqui   
1                   Real SCI           3246608  Texas Club 2025   

   Visitante_ID_Time  
0           50158297  
1            1273719

In [9]:
# Criar lista de dicion√°rios no formato desejado
confrontos_js_fase_5 = []

for _, row in df_rodadas.iterrows():
    confronto = {
        "jogo": row["Jogo"],
        "rodada": int(row["Rodada"]),
        "mandante": {
            "id": int(row["Mandante_ID_Time"]),
            "nome": row["Mandante_Nome"]
        },
        "visitante": {
            "id": int(row["Visitante_ID_Time"]),
            "nome": row["Visitante_Nome"]        }
    }
    confrontos_js_fase_5.append(confronto)

# Converter para JSON formatado
json_str = json.dumps(confrontos_js_fase_5, indent=2, ensure_ascii=False)

# Salvar como arquivo JS com uma vari√°vel global
with open("confrontos_fase5_libertadores.js", "w", encoding="utf-8") as f:
    f.write("const confrontosFase5 = ")
    f.write(json_str)
    f.write(";")

In [10]:
def exibir_confrontos(df_rodadas, rodada=None, jogo=None):

    colunas = ["Rodada", "Jogo", "Mandante_Nome", "Visitante_Nome"]
    df_filtrado = df_rodadas.copy()

    df_filtrado["Rodada"] = df_filtrado["Rodada"].astype(str) + "¬™ Rodada"    

    if rodada is not None:
        df_filtrado = df_filtrado[df_filtrado["Rodada"] == rodada]

    if jogo is not None:
        df_filtrado = df_filtrado[df_filtrado["Jogo"] == jogo]

    return df_filtrado[colunas].sort_values(by=["Jogo", "Rodada"])

In [11]:
# jogo = "Decis√£o 3¬∫ Lugar"
jogo = "Final"

# Exibir todos os confrontos do Jogo 1
display(exibir_confrontos(df_rodadas, jogo= f"{jogo}").head(8))

Rodada   Jogo              Mandante_Nome Visitante_Nome
0  17¬™ Rodada  Final  Analove10 ITAQUI GRANDE!!   L√° do Itaqui

In [12]:
def campeonato_comecou(api, ids_times):
    """Verifica se o campeonato j√° come√ßou observando a pontua√ß√£o na 1¬™ rodada."""
    for time_id in ids_times.values():
        try:
            pontuacao = api.time(time_id=time_id, rodada=17).ultima_pontuacao
            if pontuacao is not None:
                return True
        except cartolafc.errors.CartolaFCError:
            continue
    return False

def obter_pontuacao_por_rodada(api, time_id, rodada_atual):
    """Obt√©m a pontua√ß√£o do time em cada rodada at√© a rodada atual."""
    pontuacoes = {}
    for rodada in range(17, rodada_atual):
        try:
            time_rodada = api.time(time_id=time_id, rodada=rodada)
            pontuacoes[rodada] = time_rodada.ultima_pontuacao
        except cartolafc.errors.CartolaFCError as e:
            print(f"Erro ao acessar pontua√ß√£o da rodada {rodada} para o time {time_id}: {e}")
            pontuacoes[rodada] = None
    return pontuacoes



def gerar_df_pontuacoes(api, ids_times, rodada_inicio=17):
    rodada_atual = api.mercado().rodada_atual

    if not campeonato_comecou(api, ids_times):
        print("üìå A fase ainda n√£o come√ßou. Criando estrutura com placeholders.")
        df = pd.DataFrame(index=ids_times.keys(), columns=[f'Rodada {i}' for i in range(rodada_inicio, rodada_inicio + 2)])
        df[:] = None  # ou 0 se preferir
        return df

    # Campeonato come√ßou
    df = pd.DataFrame()
    for nome, time_id in ids_times.items():
        pontuacoes = obter_pontuacao_por_rodada(api, time_id, rodada_atual)
        df[nome] = pd.Series(pontuacoes)

    df = df.transpose()

    # Ajustar nome das colunas para "Rodada X"
    colunas_numeradas = df.columns.tolist()
    df.columns = [f'Rodada {i}' for i in colunas_numeradas]

    # Adicionar a linha com o time l√≠der por rodada (opcional)
    if not df.empty:
        df.loc['Lider_Rodada'] = df.idxmax()

    return df


In [13]:
# Combina os nomes dos vencedores e perdedores
nomes_completos = nomes_por_id.copy()
nomes_completos.update(nomes_perdedores)

# Agora inverte para gerar os ids_times corretamente
ids_times = {v: k for k, v in nomes_completos.items()}

df_pontuacoes = gerar_df_pontuacoes(api, ids_times)


In [14]:

if df_pontuacoes.empty:
    print("‚ö†Ô∏è Sem pontua√ß√µes dispon√≠veis ainda.")
else:
    display(df_pontuacoes.T)

Analove10 ITAQUI GRANDE!! L√° do Itaqui Real SCI Texas Club 2025  \
Rodada 17                     58.51        53.10    62.24           47.51   
Rodada 18                     78.87        87.47    79.27           98.00   
Rodada 19                     69.10        58.16    61.50           61.65   

                        Lider_Rodada  
Rodada 17                   Real SCI  
Rodada 18            Texas Club 2025  
Rodada 19  Analove10 ITAQUI GRANDE!!

In [15]:
def classificacao_por_grupo(df_rodadas, df_pontuacoes):

    df_pontuacoes_times = df_pontuacoes.drop(index='Lider_Rodada', errors='ignore')
    estatisticas = {}

    for _, confronto in df_rodadas.iterrows():
        rodada = confronto["Rodada"]
        mandante = confronto["Mandante_Nome"]
        visitante = confronto["Visitante_Nome"]
        jogo = confronto["Jogo"]
        coluna_rodada = f"Rodada {rodada}"

        if mandante not in df_pontuacoes_times.index or visitante not in df_pontuacoes_times.index:
            continue
        if coluna_rodada not in df_pontuacoes_times.columns:
            continue

        pontos_mandante = df_pontuacoes_times.at[mandante, coluna_rodada]
        pontos_visitante = df_pontuacoes_times.at[visitante, coluna_rodada]

        # Ignorar confrontos ainda n√£o disputados (com pontua√ß√£o 0 ou ausente)
        if (
            pd.isnull(pontos_mandante) or pd.isnull(pontos_visitante) or
            (pontos_mandante == 0 and pontos_visitante == 0)
        ):
            continue

        for time in [mandante, visitante]:
            if jogo not in estatisticas:
                estatisticas[jogo] = {}
            if time not in estatisticas[jogo]:
                estatisticas[jogo][time] = {
                    "Pontos": 0, "Vit√≥rias": 0, "Empates": 0, "Derrotas": 0,
                    "Total_Cartola": 0,
                    "Cartola_Sofrido": 0
                }

        # Atualizar estat√≠sticas do jogo
        estatisticas[jogo][mandante]["Total_Cartola"] += pontos_mandante
        estatisticas[jogo][mandante]["Cartola_Sofrido"] += pontos_visitante

        estatisticas[jogo][visitante]["Total_Cartola"] += pontos_visitante
        estatisticas[jogo][visitante]["Cartola_Sofrido"] += pontos_mandante

        if pontos_mandante > pontos_visitante:
            estatisticas[jogo][mandante]["Pontos"] += 3
            estatisticas[jogo][mandante]["Vit√≥rias"] += 1
            estatisticas[jogo][visitante]["Derrotas"] += 1
        elif pontos_mandante < pontos_visitante:
            estatisticas[jogo][visitante]["Pontos"] += 3
            estatisticas[jogo][visitante]["Vit√≥rias"] += 1
            estatisticas[jogo][mandante]["Derrotas"] += 1
        else:
            estatisticas[jogo][mandante]["Pontos"] += 1
            estatisticas[jogo][visitante]["Pontos"] += 1
            estatisticas[jogo][mandante]["Empates"] += 1
            estatisticas[jogo][visitante]["Empates"] += 1

    # Gerar DataFrame final
    df_resultado = pd.concat([
        pd.DataFrame({
            "Jogo": jogo,
            "Nome do Time": list(times.keys()),
            "Pontos": [stats["Pontos"] for stats in times.values()],
            "Vit√≥rias": [stats["Vit√≥rias"] for stats in times.values()],
            "Empates": [stats["Empates"] for stats in times.values()],
            "Derrotas": [stats["Derrotas"] for stats in times.values()],
            "Total Cartola": [stats["Total_Cartola"] for stats in times.values()],
            "Cartola Sofrido": [stats["Cartola_Sofrido"] for stats in times.values()],
            "Saldo Cartola": [
                stats["Total_Cartola"] - stats["Cartola_Sofrido"] for stats in times.values()
            ]
        }) for jogo, times in estatisticas.items()
    ], ignore_index=True)

    # Ordenar e adicionar posi√ß√£o
    df_resultado = df_resultado.sort_values(
        by=["Jogo", "Pontos", "Vit√≥rias", "Total Cartola", "Saldo Cartola"],
        ascending=[True, False, False, False, False]
    )

    df_resultado["Posi√ß√£o"] = df_resultado.groupby("Jogo").cumcount() + 1

    df_resultado_por_jogo = {
        jogo: df_resultado[df_resultado["Jogo"] == jogo] for jogo in df_resultado["Jogo"].unique()
    }

    return df_resultado, df_resultado_por_jogo


In [16]:
# Padroniza e exibe as pontua√ß√µes se existirem
try:
    if 'df_pontuacoes' not in globals() or df_pontuacoes.empty:
        print("üìå Nenhuma pontua√ß√£o dispon√≠vel ‚Äî provavelmente ainda n√£o h√° classificados definidos.")
    else:
        # Padroniza os nomes no df_rodadas
        df_rodadas["Mandante_Nome"] = df_rodadas["Mandante_Nome"].str.strip()
        df_rodadas["Visitante_Nome"] = df_rodadas["Visitante_Nome"].str.strip()

        # Padroniza os √≠ndices do df_pontuacoes
        df_pontuacoes.index = df_pontuacoes.index.str.strip()

        # Exibe
        display(df_pontuacoes)

except NameError as e:
    print("‚ùå Vari√°vel df_rodadas ou df_pontuacoes n√£o est√° definida ainda.")
except Exception as e:
    print(f"‚ùå Erro inesperado: {e}")


Rodada 17        Rodada 18  \
Analove10 ITAQUI GRANDE!!     58.51            78.87   
L√° do Itaqui                  53.10            87.47   
Real SCI                      62.24            79.27   
Texas Club 2025               47.51            98.00   
Lider_Rodada               Real SCI  Texas Club 2025   

                                           Rodada 19  
Analove10 ITAQUI GRANDE!!                      69.10  
L√° do Itaqui                                   58.16  
Real SCI                                       61.50  
Texas Club 2025                                61.65  
Lider_Rodada               Analove10 ITAQUI GRANDE!!

In [17]:
try:
    # Verifica se as vari√°veis existem e est√£o v√°lidas
    if (
        'df_rodadas' not in globals() or df_rodadas.empty or
        'df_pontuacoes' not in globals() or df_pontuacoes.empty
    ):
        print("‚ö†Ô∏è Rodadas ou pontua√ß√µes ainda n√£o dispon√≠veis ‚Äî classifica√ß√£o n√£o ser√° gerada.")
    
    elif 'classificacao_por_grupo' not in globals():
        print("‚ö†Ô∏è Fun√ß√£o 'classificacao_por_grupo' n√£o est√° definida.")
    
    else:
        # Gerar a classifica√ß√£o da fase 5
        df_resultado, df_resultado_por_jogo = classificacao_por_grupo(
            df_rodadas, df_pontuacoes
        )

        # Salvar cada grupo em uma aba do Excel
        with pd.ExcelWriter("classificacao_por_grupo_fase_5.xlsx") as writer:
            for jogo, df in df_resultado_por_jogo.items():
                df.to_excel(writer, sheet_name=jogo, index=False)

        # Exibir a classifica√ß√£o geral
        df_resultado_jogo_1 = df_resultado[df_resultado["Jogo"] == "Final"]
        df_resultado_jogo_2 = df_resultado[df_resultado["Jogo"] == "Decis√£o 3¬∫ Lugar"]

        display(df_resultado_jogo_1, df_resultado_jogo_2)

except Exception as e:
    print(f"‚ùå Erro ao gerar a classifica√ß√£o: {e}")


Jogo               Nome do Time  Pontos  Vit√≥rias  Empates  Derrotas  \
0  Final  Analove10 ITAQUI GRANDE!!       3         1        0         0   
1  Final               L√° do Itaqui       0         0        0         1   

   Total Cartola  Cartola Sofrido  Saldo Cartola  Posi√ß√£o  
0          58.51            53.10           5.41        1  
1          53.10            58.51          -5.41        2

Jogo     Nome do Time  Pontos  Vit√≥rias  Empates  Derrotas  \
2  Decis√£o 3¬∫ Lugar         Real SCI       3         1        0         0   
3  Decis√£o 3¬∫ Lugar  Texas Club 2025       0         0        0         1   

   Total Cartola  Cartola Sofrido  Saldo Cartola  Posi√ß√£o  
2          62.24            47.51          14.73        1  
3          47.51            62.24         -14.73        2

In [18]:
# Verifica se a vari√°vel df_resultado_por_grupo existe e est√° populada
if 'df_resultado_por_jogo' in locals() and df_resultado_por_jogo:
    # Criar estrutura em formato de dicion√°rio para JSON/JS
    classificacao_js_fase_5 = {}

    for jogo, df in df_resultado_por_jogo.items():
        classificacao_js_fase_5[jogo] = []

        # Remove duplicatas
        df = df.drop_duplicates(subset=["Nome do Time"])

        for _, row in df.iterrows():
            classificacao_js_fase_5[jogo].append({
                "posicao": int(row["Posi√ß√£o"]),
                "nome": row["Nome do Time"],
                "pontos": int(row["Pontos"]),
                "vitorias": int(row["Vit√≥rias"]),
                "empates": int(row["Empates"]),
                "derrotas": int(row["Derrotas"]),
                "totalCartola": float(row["Total Cartola"]),
                "cartolaSofrido": float(row["Cartola Sofrido"]),
                "saldoCartola": float(row["Saldo Cartola"])
            })


    # Converter para JSON formatado
    json_str = json.dumps(classificacao_js_fase_5, indent=2, ensure_ascii=False)

    # Salvar como arquivo JS com uma vari√°vel global
    with open("classificacao_por_grupo_fase_5.js", "w", encoding="utf-8") as f:
        f.write("const classificacaoFase5 = ")
        f.write(json_str)
        f.write(";")

    print("‚úÖ Arquivo JS salvo com sucesso.")

else:
    print("‚ö†Ô∏è Classifica√ß√£o da fase 5 indispon√≠vel. Criando placeholders.")

    # ‚ö†Ô∏è Classifica√ß√£o da fase 5 indispon√≠vel ‚Äî criar estrutura zerada manualmente

    # Dicion√°rio com confrontos definidos
    confrontos_fase_5 = {
        "Final": ["Analove10 ITAQUI GRANDE!!", "L√° do Itaqui"],
        "Decis√£o 3¬∫ Lugar": ["Real SCI", "Texas Club 2025"]
    }

    # Estrutura zerada para o JS
    classificacao_js_fase_5 = {}

    for jogo, times in confrontos_fase_5.items():
        classificacao_js_fase_5[jogo] = []
        for nome_time in times:
            classificacao_js_fase_5[jogo].append({
                "posicao": 0,
                "nome": nome_time,
                "pontos": 0,
                "vitorias": 0,
                "empates": 0,
                "derrotas": 0,
                "totalCartola": 0.0,
                "cartolaSofrido": 0.0,
                "saldoCartola": 0.0
            })

    # Salvar como JS
    with open("classificacao_por_grupo_fase_5.js", "w", encoding="utf-8") as f:
        f.write("const classificacaoFase5 = ")
        json.dump(classificacao_js_fase_5, f, indent=2, ensure_ascii=False)
        f.write(";")

    print("‚úÖ Arquivo JS com classifica√ß√£o zerada gerado com sucesso.")


‚úÖ Arquivo JS salvo com sucesso.


In [19]:
def exibir_resultados_rodada(df_rodadas, df_pontuacoes, rodada, jogo=None):
    """
    Exibe os resultados de uma rodada espec√≠fica, com pontua√ß√£o e dados dos times.
    """

    if rodada not in df_rodadas["Rodada"].values:
        return pd.DataFrame([{
            "Jogo": jogo or "-",
            "Rodada": rodada,
            "Mandante_Nome": "-",          
            "Mandante_Pontos": "-",
            "Visitante_Nome": "-",           
            "Visitante_Pontos": "-",
        }])

    df_filtrado = df_rodadas[df_rodadas["Rodada"] == rodada]
    if jogo:
        df_filtrado = df_filtrado[df_filtrado["Jogo"] == jogo]

    resultados = []

    for _, row in df_filtrado.iterrows():
        jogo_ = row["Jogo"]
        mandante = row["Mandante_Nome"]
        visitante = row["Visitante_Nome"]

        # pontos_mandante = df_pontuacoes.get(f"Rodada {rodada}", {}).get(mandante, None)
        # pontos_visitante = df_pontuacoes.get(f"Rodada {rodada}", {}).get(visitante, None)

        pontos_mandante = df_pontuacoes.loc[mandante, f"Rodada {rodada}"] if mandante in df_pontuacoes.index and f"Rodada {rodada}" in df_pontuacoes.columns else None
        pontos_visitante = df_pontuacoes.loc[visitante, f"Rodada {rodada}"] if visitante in df_pontuacoes.index and f"Rodada {rodada}" in df_pontuacoes.columns else None


        resultados.append({
            "Jogo": jogo_,
            "Rodada": rodada,
            "Mandante_Nome": mandante,
            "Mandante_Pontos": pontos_mandante,
            "Visitante_Nome": visitante,
            "Visitante_Pontos": pontos_visitante
        })

    return pd.DataFrame(resultados)


In [20]:
# Exibir resultados da 15¬™ rodada
df_resultados_rodada_15 = exibir_resultados_rodada(df_rodadas, df_pontuacoes, rodada=17)

# # Exibir apenas os resultados do Grupo B na 1¬™ rodada
# df_resultados_jogo_1 = exibir_resultados_rodada(df_rodadas, df_pontuacoes, rodada=17, jogo="Decis√£o 3¬∫ Lugar")

# Exibir
display(df_resultados_rodada_15)

Jogo  Rodada              Mandante_Nome  Mandante_Pontos  \
0             Final      17  Analove10 ITAQUI GRANDE!!            58.51   
1  Decis√£o 3¬∫ Lugar      17                   Real SCI            62.24   

    Visitante_Nome  Visitante_Pontos  
0     L√° do Itaqui             53.10  
1  Texas Club 2025             47.51

In [21]:
# Criar arquivo com uma aba para cada rodada contendo os resultados detalhados
from pathlib import Path

# Caminho do arquivo de sa√≠da
caminho_resultados = "resultados_fase_5.xlsx"

# Descobrir as rodadas √∫nicas no DataFrame
rodadas_disponiveis = sorted(df_rodadas["Rodada"].unique())

with pd.ExcelWriter(caminho_resultados) as writer:
    for rodada in rodadas_disponiveis:
        df_resultados = exibir_resultados_rodada(df_rodadas, df_pontuacoes, rodada=rodada)
        nome_aba = f"Rodada {rodada}"
        df_resultados.to_excel(writer, sheet_name=nome_aba, index=False)

print(f"Arquivo salvo com sucesso: {Path(caminho_resultados).resolve()}")

display(df_resultados)

Arquivo salvo com sucesso: C:\Users\ferna\Projetos\GitHub\cartola_fbc\libertadores\datasets_liberta\resultados_fase_5.xlsx


Jogo  Rodada              Mandante_Nome  Mandante_Pontos  \
0             Final      17  Analove10 ITAQUI GRANDE!!            58.51   
1  Decis√£o 3¬∫ Lugar      17                   Real SCI            62.24   

    Visitante_Nome  Visitante_Pontos  
0     L√° do Itaqui             53.10  
1  Texas Club 2025             47.51

In [22]:
resultados_js = []

for rodada in sorted(df_rodadas["Rodada"].unique()):
    df_resultados = exibir_resultados_rodada(df_rodadas, df_pontuacoes, rodada=rodada)
    
    for _, row in df_resultados.iterrows():
        resultado = {
            "jogo": row["Jogo"],
            "rodada": int(rodada),
            "mandante": {
                "nome": row["Mandante_Nome"],
                "pontos": float(row["Mandante_Pontos"]) if row["Mandante_Pontos"] is not None else None
            },
            "visitante": {
                "nome": row["Visitante_Nome"],
                "pontos": float(row["Visitante_Pontos"]) if row["Visitante_Pontos"] is not None else None               
            },
            "vencedor": (
                "mandante" if row["Mandante_Pontos"] is not None and row["Visitante_Pontos"] is not None and row["Mandante_Pontos"] > row["Visitante_Pontos"]
                else "visitante" if row["Mandante_Pontos"] is not None and row["Visitante_Pontos"] is not None and row["Mandante_Pontos"] < row["Visitante_Pontos"]
                else "empate" if row["Mandante_Pontos"] == row["Visitante_Pontos"] and row["Mandante_Pontos"] is not None
                else "indefinido"
            )

        }
        resultados_js.append(resultado)


# Exportar para arquivo .js
import json

with open("resultados_fase_5.js", "w", encoding="utf-8") as f:
    f.write("const resultadosFase5 = ")
    f.write(json.dumps(resultados_js, indent=2, ensure_ascii=False))
    f.write(";")


### Identificando Vencedores das Semifinais

In [23]:
def obter_classificados_com_id(resultados, ids_por_nome):
    classificados = []
    jogos_agrupados = {}
    for jogo in resultados:
        chave = jogo['jogo']
        if chave not in jogos_agrupados:
            jogos_agrupados[chave] = []
        jogos_agrupados[chave].append(jogo)

    for chave, partidas in jogos_agrupados.items():
        if len(partidas) < 2:
            continue

        partidas = sorted(partidas, key=lambda x: x['rodada'])
        ida, volta = partidas

        if ida['mandante']['pontos'] is None or volta['mandante']['pontos'] is None:
            continue

        time1 = ida['mandante']['nome']
        time2 = ida['visitante']['nome']

        pontos_time1 = ida['mandante']['pontos'] + volta['visitante']['pontos']
        pontos_time2 = ida['visitante']['pontos'] + volta['mandante']['pontos']

        if pontos_time1 > pontos_time2:
            vencedor = time1
        elif pontos_time2 > pontos_time1:
            vencedor = time2
        else:
            vencedor = "EMPATE"

        classificados.append({
            "jogo": chave,
            "classificado_nome": vencedor,
            "classificado_id": ids_por_nome.get(vencedor) if vencedor in ids_por_nome else None
        })

    return classificados


In [24]:
# Inverter os nomes
ids_por_nome = {v: k for k, v in nomes_por_id.items()}

# Ler o resultados_fase_5.js (como j√° fizemos antes)
with open("resultados_fase_5.js", "r", encoding="utf-8") as f:
    conteudo = f.read()

conteudo_json = conteudo.replace("const resultadosFase5 = ", "").strip().rstrip(";")
resultados = json.loads(conteudo_json)

# Obter os classificados
classificados = obter_classificados_com_id(resultados, ids_por_nome)

# Salvar
df_classificados = pd.DataFrame(classificados)
df_classificados.to_excel("classificados_fase_5.xlsx", index=False)

with open("classificados_fase_5.js", "w", encoding="utf-8") as f:
    f.write("const classificadosFase5 = ")
    json.dump(classificados, f, ensure_ascii=False, indent=2)
    f.write(";")
print("‚úÖ Classificados salvos com sucesso em 'classificados_fase_5.xlsx' e 'classificados_fase_5.js'.")

# Exibir para validar
print("Classificados encontrados:", classificados)


‚úÖ Classificados salvos com sucesso em 'classificados_fase_5.xlsx' e 'classificados_fase_5.js'.
Classificados encontrados: []


### Pontua√ß√£o dos Campe√µes Libertadores

In [25]:
# import json
# import pandas as pd

# # Pontua√ß√£o por coloca√ß√£o
# pontuacao_por_posicao = {1: 120, 2: 80, 3: 50, 4: 30}

# # Fun√ß√£o para ordenar os finalistas corretamente
# def gerar_ranking_libertadores(resultados_fase_5, ids_por_nome):
#     classificados = obter_classificados_com_id(resultados_fase_5, ids_por_nome)

#     # Cria um dicion√°rio para mapear o nome dos classificados por confronto
#     classificados_por_jogo = {c["jogo"]: c["classificado_nome"] for c in classificados}

#     # Identifica final e semifinal
#     finais = [j for j in classificados if j["jogo"].startswith("final")]
#     semis = [j for j in classificados if j["jogo"].startswith("semi")]

#     if len(finais) != 1 or len(semis) != 2:
#         raise ValueError("N√∫mero inesperado de finais ou semifinais.")

#     final = finais[0]
#     semi1, semi2 = semis

#     campeao = final["classificado_nome"]
#     vice = [t for t in [semi1["classificado_nome"], semi2["classificado_nome"]] if t != campeao][0]
#     perdedores = [t for t in [semi1["classificado_nome"], semi2["classificado_nome"]] if t != campeao and t != vice]

#     # Buscar nomes dos perdedores nas semifinais
#     semifinalistas = set([semi1["classificado_nome"], semi2["classificado_nome"]])
#     perdedor1 = [t for t in semifinalistas if t not in [campeao, vice]][0]
#     perdedor2 = [t for t in [ida['mandante']['nome'], ida['visitante']['nome']] if t not in semifinalistas][0]

#     # Classifica√ß√£o final
#     classificacao_final = [
#         {"time": campeao, "pontos": pontuacao_por_posicao[1], "posicao": 1},
#         {"time": vice,    "pontos": pontuacao_por_posicao[2], "posicao": 2},
#         {"time": perdedor1, "pontos": pontuacao_por_posicao[3], "posicao": 3},
#         {"time": perdedor2, "pontos": pontuacao_por_posicao[4], "posicao": 4},
#     ]

#     return classificacao_final

# # Supondo que voc√™ j√° tenha carregado:
# # - resultados_fase_5
# # - ids_por_nome

# # Carregamento dos dados (ajuste caminho conforme necess√°rio)
# with open("resultados_fase_5.js", "r", encoding="utf-8") as f:
#     conteudo = f.read()
# conteudo_json = conteudo.replace("const resultadosFase5 = ", "").strip().rstrip(";")
# resultados_fase_5 = json.loads(conteudo_json)

# # Inverter os nomes (ajuste conforme seu dicion√°rio real)
# ids_por_nome = {v: k for k, v in nomes_por_id.items()}

# # Gerar ranking
# ranking_libertadores = gerar_ranking_libertadores(resultados_fase_5, ids_por_nome)

# # Montar formato para JS
# ranking_libertadores_js = []
# for item in ranking_libertadores:
#     ranking_libertadores_js.append({
#         "time": item["time"],
#         "pontos": item["pontos"],
#         "aparicoes": 1,
#         "posicoes": [{"mes": "Libertadores", "posicao": item["posicao"]}]
#     })

# # Salvar em JSON estilo JS
# with open("../../campeoes/datasets_campeoes/ranking_libertadores.js", "w", encoding="utf-8") as f:
#     f.write("const rankingCampeoes = {\n  \"Libertadores\": ")
#     json.dump(ranking_libertadores_js, f, indent=2, ensure_ascii=False)
#     f.write("\n};")
#     print("‚úÖ Arquivo ranking_libertadores.js salvo com sucesso!")
